In [2]:
import json
import numpy as np
from pprint import pprint
from os import listdir
from os.path import isfile, join
from MongoOps import MongoOps
from ICSParser import ICSParser
from make_directions_matrix import DirectionsMatrix
import random

from OpenHouseGraph import *

In [31]:
locations = []
mops = MongoOps()
sample_data_files = [f for f in listdir("/data") if isfile(join("/data", f))]
for open_house_file in sample_data_files:
    try:
        parser = ICSParser("/data/%s" % open_house_file)
        event = parser.to_dict()
        # print(event)
        result = mops.safe_query_for_location_info(event)
        locations += [result]
    except:
        pass

random_locations = random_combination(locations, 7)
DM = DirectionsMatrix(random_locations, mops)
DM.generate_simplified_directions_matrix()
sdm = DM.simplified_directions_matrix


In [32]:
# Showing off EST/EDT time of day of the open houses and the conversion to minutes from midnight that day
for i in range(len(sdm)):
    start = int(sdm[i]['dtstart'][9:-3])-600
    start_minutes = 6./10*start
    end = int(sdm[i]['dtend'][9:-3])-600
    end_minutes = 6./10*end
    start = sdm[i]['start'] = start
    start_minutes = sdm[i]['start_minutes'] = start_minutes
    end = sdm[i]['end'] = end
    end_minutes = sdm[i]['end_minutes'] = end_minutes
vertices = []
V = None
for i in range(len(sdm)):
    V = {'ID' : i, 
            'start' : sdm[i]['start_minutes'], 
            'end' : sdm[i]['end_minutes'], 
            'edges' : sdm[i]['durations'],
            'address_hash' : sdm[i]['address_hash'],
            'address' : sdm[i]['location']['address']}
    vertices += [V]


In [33]:
ohg = OpenHouseGraph(vertices)

In [34]:
paths = []
for v in ohg.vertices():
    starting_id = v['ID']
    starting_vertex = ohg.get_vertex_from_vid(starting_id)
    start_time = starting_vertex['start']
    path = ohg.visit_next(starting_vertex, start_time, visited=[])
    if len(path) > 0:
        for p in path:
            if p not in paths:
                paths += [p]
paths = np.array(paths)

max_len = max([len(path) for path in paths])
print(max_len)
P = []
for path in paths:
    if len(path) >= max_len:
        P += [path]
print(P)

[0, 1, 2, 4, 5]
[0, 1, 2, 4, 5]
[1, 0, 2, 4, 5]
[1, 0, 2, 4, 5, 6]
[3, 0, 1, 2, 4, 5]
[3, 0, 1, 2, 4, 5]
[3, 0, 1, 2, 4, 5, 6]
[6, 0, 2, 4, 5]
[6, 0, 2, 4, 5]
7
[[1, 0, 2, 4, 5, 6, 3], [3, 0, 1, 2, 4, 5, 6]]


In [35]:
print(vertices)
for path in P:
    print('------------------------- Showing path for {}'.format(path))
    for p in path:
        print(vertices[p]['address'])
    print('\n')

[{'ID': 0, 'start': 558.0, 'end': 660.0, 'edges': [[1, 15.28], [2, 13.67], [3, 23.51], [4, 54.0], [5, 18.09], [6, 41.01]], 'address_hash': b'\xef\x94\xa6\xe9\x87\xfe|\x84O_\xb1\x12\xe4W7@\xff\x96X/', 'address': '3720 oxford rd new hartford ny 13413'}, {'ID': 1, 'start': 540.0, 'end': 618.0, 'edges': [[0, 16.64], [2, 11.53], [3, 29.44], [4, 56.8], [5, 15.95], [6, 52.96]], 'address_hash': b'W\xa1Yd\x94\xc1\xabA\xc5\xb0G\x96\xbe?~\xc3P\xfcPg', 'address': '900 lansing st utica ny 13501'}, {'ID': 2, 'start': 660.0, 'end': 780.0, 'edges': [[0, 14.91], [1, 10.01], [3, 26.35], [4, 52.71], [5, 8.43], [6, 49.89]], 'address_hash': b'x\tO\x04\xce\xf7W!\xebCW\x1b\x1e\x85\x11\xb3\xe0\x02\xa1Y', 'address': '5516 sunrise ter marcy ny 13403'}, {'ID': 3, 'start': 480.0, 'end': 600.0, 'edges': [[0, 23.26], [1, 27.5], [2, 25.87], [4, 33.41], [5, 24.77], [6, 39.46]], 'address_hash': b'+\xc7\xfa\x8b\xc0u#\xfa\xfd\xdf\xearJ\xc7\xb42\xd7\xd1\x93\xe8', 'address': '5668 youngs rd vernon ny 13476'}, {'ID': 4, 's